In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, LeakyReLU
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time


In [2]:
EPOCH_COUNT = 20

NAME = f"single_rgb_image_regression_V03_epochs_{EPOCH_COUNT}_mit_Sigmoid_{int(time.time())}"


In [3]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
# def create_data_via_ImageDataGenerator():
#     train = ImageDataGenerator(rescale=1/255)
#     validation = ImageDataGenerator(rescale=1/255)
#     train_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                               target_size=(60, 60), batch_size=3, class_mode='binary')
#     validation_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                                target_size=(60, 60), batch_size=3, class_mode='binary')


In [3]:
import numpy as np
def load_data_via_numpy_load():
    return (np.load("CNN_trainset_single_RGB_regression_V03_X.npy")/255, np.load("CNN_trainset_single_RGB_regression_V03_Y.npy"))


In [4]:
X, Y = load_data_via_numpy_load()


In [7]:
X[0]


array([[[0.38431373, 0.38431373, 0.43137255],
        [0.3372549 , 0.3254902 , 0.36078431],
        [0.35686275, 0.34901961, 0.38431373],
        ...,
        [0.29803922, 0.37647059, 0.36470588],
        [0.33333333, 0.42745098, 0.41176471],
        [0.35686275, 0.45882353, 0.43529412]],

       [[0.33333333, 0.3254902 , 0.3372549 ],
        [0.35294118, 0.3372549 , 0.35294118],
        [0.36078431, 0.35294118, 0.36470588],
        ...,
        [0.34901961, 0.40392157, 0.39215686],
        [0.36078431, 0.42745098, 0.40784314],
        [0.41960784, 0.48235294, 0.4627451 ]],

       [[0.55686275, 0.49803922, 0.51764706],
        [0.45490196, 0.40392157, 0.42352941],
        [0.39215686, 0.37254902, 0.39215686],
        ...,
        [0.35294118, 0.40784314, 0.40392157],
        [0.37647059, 0.45098039, 0.45098039],
        [0.41176471, 0.48627451, 0.49019608]],

       ...,

       [[0.34509804, 0.33333333, 0.34901961],
        [0.30980392, 0.30588235, 0.31764706],
        [0.24705882, 0

In [8]:
Y.tolist().count(0)

48000

In [9]:
X = X[0:10000]
Y = Y[0:10000]

X.shape

(10000, 60, 60, 3)

In [10]:


X = X.reshape(-1, 60, 60, 3)

X.shape


(10000, 60, 60, 3)

In [11]:
def build_model():
    model = Sequential()

    # 1st Conv-Layer
    model.add(Conv2D(128, (3, 3), input_shape=(X.shape[1:])))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))

    # 2nd Conv-Layer
    model.add(Conv2D(192, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    # 3rd Conv-Layer
    model.add(Conv2D(320, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # 4th Conv-Layer
    model.add(Conv2D(576, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())

    # 1st Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 2nd Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 3rd Fully-Connected
    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    return model


model = build_model()


In [12]:

model.compile(loss="mse", optimizer="adam", metrics=["mse"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 128)       3584      
_________________________________________________________________
activation (Activation)      (None, 58, 58, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 58, 58, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 192)       221376    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 192)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 192)       7

In [13]:
model.fit(X, Y, batch_size=32, epochs=EPOCH_COUNT, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/20
282/282 [==============================] - 50s 61ms/step - loss: 0.2007 - mse: 0.2007 - val_loss: 0.2034 - val_mse: 0.2034
Epoch 2/20
282/282 [==============================] - 13s 47ms/step - loss: 0.4703 - mse: 0.4703 - val_loss: 0.5284 - val_mse: 0.5284
Epoch 3/20
282/282 [==============================] - 13s 47ms/step - loss: 0.4081 - mse: 0.4081 - val_loss: 0.3974 - val_mse: 0.3974
Epoch 4/20
282/282 [==============================] - 13s 47ms/step - loss: 0.3523 - mse: 0.3523 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 5/20
282/282 [==============================] - 13s 47ms/step - loss: 0.2368 - mse: 0.2368 - val_loss: 0.2094 - val_mse: 0.2094
Epoch 6/20
282/282 [==============================] - 13s 47ms/step - loss: 0.2371 - mse: 0.2371 - val_loss: 0.2094 - val_mse: 0.2094
Epoch 7/20
282/282 [==============================] - 13s 47ms/step - loss: 0.2371 - mse: 0.2371 - val_loss: 0.2094 - val_mse: 0.2094
Epoch 8/20
282/282 [==============================] - 13s 47ms

# Test


In [14]:
# model = tf.keras.models.load_model("./models/single_rgb_image_big_dataset")


In [15]:
# # Alias

# img1 = Image.open(
#     "D:\Main\MA_PROGR\Data\Train\LED_Wand_Aufnahmen\Alias\LED_Wand_20001.png")
# img1 = np.asarray(img1)/255




# img1 = img1.reshape(1,60,60,3)
# print(img1.shape)
# # print(img1)
# res1 = model.predict(img1)
# print(res1)


In [16]:
# # NoAlias

# img2 = Image.open(
#     "D:\\Main\\MA_PROGR\\Data\\Train\\LED_Wand_Aufnahmen\\NoAlias\\abstract_cross_203.png")
# img2 = np.asarray(img2)/255

# img2 = img2.reshape(1, 60, 60, 3)
# img2.shape
# # print(img2)
# res2 = model.predict(img2)
# print(res2)


In [17]:
# import matplotlib.pyplot as plt
# plt.imshow(img2.reshape(60, 60, 3))


In [18]:
tf.keras.models.save_model(model, "./models/{}".format(NAME))

INFO:tensorflow:Assets written to: ./models/single_rgb_image_regression_V03_epochs_20_mit_Sigmoid_1653661303\assets


INFO:tensorflow:Assets written to: ./models/single_rgb_image_regression_V03_epochs_20_mit_Sigmoid_1653661303\assets
